In [1]:
import pandas as pd
import pyreadr
import numpy as np

In [2]:
#load features data
features = pd.read_csv('../FinalData/data_features_10072023.csv')
features.iloc[:,1:]
features.shape

/tmp/ipykernel_3052/1120385982.py:2: DtypeWarning: Columns (26,27,29,30,33,34) have mixed types. Specify dtype option on import or set low_memory=False.
  features = pd.read_csv('../FinalData/data_features_10072023.csv')


(732472, 54)

In [3]:
#add paractice information
practice = pyreadr.read_r('../ServerData_13Oct2020/d_practice.Rdata')
practice = practice['d_practice']
features = features.merge(practice[['practice_id','postcode_district','imd_decile','Country','County','LocalAuthority','OutputAreaClassification']], on='practice_id',how='left')
features.reset_index(drop=True, inplace=True)
features.shape

/opt/conda/envs/rapids/lib/python3.10/site-packages/pyreadr/_pyreadr_parser.py:233: RuntimeWarning: invalid value encountered in cast
  df[colname] = df[colname].values.astype("datetime64[D]").astype(datetime)


(732472, 60)

In [4]:
#BASE DATA
#extract patients from England, Wales, Scotland
features = features[(features.Country == 'England') |(features.Country == 'Wales') | (features.Country == 'Scotland')]
print(features.shape)
print('train :', features[features.Country == 'England'].shape[0])
print('val :', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0])

(724438, 60)
train : 697668
val : 26770


In [5]:
# SEX drop NA
temp = features.dropna(subset=('sex'))
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features.reset_index(drop=True, inplace=True)

new N:  720611
diff:  3827
train : 693841
val : 26770
diff train:  3827
diff val:  0


In [6]:
# Include only 18+
temp = features[features.age >= 18]
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features.reset_index(drop=True, inplace=True)

new N:  583370
diff:  137241
train : 561100
val : 22270
diff train:  132741
diff val:  4500


In [7]:
print('Missing BMI: ', features[features.height.isna()].shape[0])

Missing BMI:  48186


In [8]:
# BMI include only 15-75 and code NA as 0 (not recorded)
temp = features[((features.BMI >= 15) & (features.BMI <= 75))|(features.BMI.isna())]
temp['BMI'][temp.BMI.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp
features = temp
features.reset_index(drop=True, inplace=True)

new N:  578096
diff:  5274


/tmp/ipykernel_3052/1773957297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['BMI'][temp.BMI.isna()] = 0
/tmp/ipykernel_3052/1773957297.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp['BMI'][temp.BMI.isna()] = 0


train : 555964
val : 22132
diff train:  5136
diff val:  138


In [9]:
# replace NA in ethnicity and ethnic_group with 'not_recorded'
features.ethnicity[features.ethnic_group.isna()] = 'not_recorded'
features.ethnic_group[features.ethnic_group.isna()] = 'not_recorded'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [10]:
#for smoking, anyone with no record, consider them non-smoker
features.smokingStatus[features.smokingStatus.isna()] = 'never'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [11]:
#for PEF, re-assign those with no data as a 'not_recorded'
features.PEFStatus[features.PEFStatus.isna()] = 'not_recorded'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [12]:
#for Eosinophil, re-assign those with no data as a 'unknown'
features.EosinophilLevel[features.EosinophilLevel.isna()] = 'unknown'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [13]:
#for Device type, re-assign those with no data as a 'unknown'
features.DeviceType[features.DeviceType.isna()] = 'unknown'
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [14]:
#BTS Step (if none recorded, it means that they are at step 0
features.BTS_step[features.BTS_step.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [15]:
#do the same thing for other continuous vars
features.average_daily_dose_ICS[features.average_daily_dose_ICS.isna()] = 0
features.prescribed_daily_dose_ICS[features.prescribed_daily_dose_ICS.isna()] = 0
features.ICS_medication_possesion_ratio[features.ICS_medication_possesion_ratio.isna()] = 0
features.Spacer[features.Spacer.isna()] = 0
features.numOCS[features.numOCS.isna()] = 0
features.numPCS[features.numPCS.isna()] = 0
features.numAntibioticsEvents[features.numAntibioticsEvents.isna()] = 0
features.numAntibioticswithLRTI[features.numAntibioticswithLRTI.isna()] = 0
features.numOCSEvents[features.numOCSEvents.isna()] = 0
features.numOCSwithLRTI[features.numOCSwithLRTI.isna()] = 0
features.numAsthmaAttacks[features.numAsthmaAttacks.isna()] = 0
features.numAcuteRespEvents[features.numAcuteRespEvents.isna()] = 0
features.PriorEducation[features.PriorEducation.isna()] = 0
features.numPCSAsthma[features.numPCSAsthma.isna()] = 0
features.CharlsonScore[features.CharlsonScore.isna()] = 0
features.numHospEvents[features.numHospEvents.isna()] = 0
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])

new N:  578096
diff:  0


In [16]:
features.columns

Index(['Unnamed: 0', 'patid', 'practice_id', 'sex', 'age', 'BMI', 'weight',
       'height', 'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'postcode_district',

## Categorized cont vars

In [17]:
#BMI
def cat_bmi(x):
    if (x==0):
        return 'not recorded'
    elif (x>=15) & (x<18.5):
        return 'underweight'
    elif (x>=18.5) & (x<25):
        return 'normal'
    elif (x>=25) & (x<30):
        return 'overweight'
    elif (x>=30):
        return 'obese'
    
    
features['cat_BMI'] = features.BMI.apply(lambda x: cat_bmi(x))
features.shape

(578096, 61)

In [18]:
#Age
def cat_age(x):
    if (x>=18) & (x<35):
        return '18-34'
    elif (x>=35) & (x<55):
        return '35-54'
    elif (x>=55) & (x<75):
        return '55-74'
    elif (x>=75):
        return '>=75'
    
features['cat_age'] = features.age.apply(lambda x: cat_age(x))
features.shape

(578096, 62)

In [19]:
#Smoking
def cat_smoking(x):
    if (x=='never'):
        return 'Non Smoker'
    elif (x=='former'):
        return 'Former Smoker'
    elif (x=='current'):
        return 'Active Smoker'
features['smokingStatus'] = features.smokingStatus.apply(lambda x: cat_smoking(x))
features.shape

(578096, 62)

In [20]:
#Average daily ICS
def cat_ics(x):
    if (x==0):
        return '0'
    elif (x>=1) & (x<=200):
        return '1-200'
    elif (x>200) & (x<=400):
        return '201-400'
    elif (x>400):
        return '>400'
    
features['cat_average_daily_dose_ICS'] = features.average_daily_dose_ICS.apply(lambda x: cat_ics(x))
features['cat_prescribed_daily_dose_ICS'] = features.prescribed_daily_dose_ICS.apply(lambda x: cat_ics(x))
features.shape


(578096, 64)

In [21]:
#clean ICS medication ratio
temp = features.drop(features[features.ICS_medication_possesion_ratio==np.inf].index)
print('new N: ', temp.shape[0])
print('diff: ', features.shape[0] - temp.shape[0])
print('train :', temp[temp.Country == 'England'].shape[0])
print('val :', temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
print('diff train: ', features[features.Country == 'England'].shape[0] - temp[temp.Country == 'England'].shape[0])
print('diff val: ', features[(features.Country == 'Wales') | (features.Country == 'Scotland')].shape[0] - temp[(temp.Country == 'Wales') | (temp.Country == 'Scotland')].shape[0])
features = temp


new N:  577962
diff:  134
train : 555836
val : 22126
diff train:  128
diff val:  6


In [22]:
#cat ICS medication ratio
def cat_ics_medication(x):
    if x==0:
        return 'No ICS Prescribed'
    elif (x>0) & (x<0.40):
        return '>0%-39.9%'
    elif (x>=0.40) & (x<.60):
        return '40%-59.9%'
    elif (x>0.60) :
        return '>=60%'
    
features['cat_ICS_medication_possesion_ratio'] = features.ICS_medication_possesion_ratio.apply(lambda x: cat_ics_medication(x))
features.shape


(577962, 65)

In [23]:
#cat other continuous vars
def cat_continuous(x):
    if x==0:
        return '0'
    elif x==1:
        return '1'
    elif x==2:
        return '2'
    elif (x>2) :
        return '>=3'
    
features['cat_numOCS'] = features.numOCS.apply(lambda x: cat_continuous(x))
features['cat_numOCSEvents'] = features.numOCSEvents.apply(lambda x: cat_continuous(x))
features['cat_numOCSwithLRTI'] = features.numOCSwithLRTI.apply(lambda x: cat_continuous(x))
features['cat_numAcuteRespEvents'] = features.numAcuteRespEvents.apply(lambda x: cat_continuous(x))
features['cat_numAntibioticsEvents'] = features.numAntibioticsEvents.apply(lambda x: cat_continuous(x))
features['cat_numAntibioticswithLRTI'] = features.numAntibioticswithLRTI.apply(lambda x: cat_continuous(x))
features['cat_numAsthmaAttacks'] = features.numAsthmaAttacks.apply(lambda x: cat_continuous(x))
features['cat_numHospEvents'] = features.numHospEvents.apply(lambda x: cat_continuous(x))
features['cat_numPCS'] = features.numPCS.apply(lambda x: cat_continuous(x))
features['cat_numPCSAsthma'] = features.numPCSAsthma.apply(lambda x: cat_continuous(x))
features.shape

(577962, 75)

In [24]:
features.columns

Index(['Unnamed: 0', 'patid', 'practice_id', 'sex', 'age', 'BMI', 'weight',
       'height', 'ethnicity', 'ethnic_group', 'smokingStatus', 'CharlsonScore',
       'count_rhinitis', 'count_cardiovascular', 'count_heartfailure',
       'count_psoriasis', 'count_anaphylaxis', 'count_diabetes', 'count_ihd',
       'count_anxiety', 'count_eczema', 'count_nasalpolyps',
       'count_paracetamol', 'count_nsaids', 'count_betablocker', 'id',
       'event_date', 'recorded_date', 'visit_id', 'code_id', 'snomed_id',
       'numeric_1', 'numeric_2', 'created_datetime', 'updated_datetime',
       'PEFStatus', 'EosinophilLevel', 'BTS_step', 'average_daily_dose_ICS',
       'prescribed_daily_dose_ICS', 'ICS_medication_possesion_ratio',
       'DeviceType', 'Spacer', 'numOCS', 'PriorEducation', 'numPCS',
       'numPCSAsthma', 'numAntibioticsEvents', 'numAntibioticswithLRTI',
       'numOCSEvents', 'numOCSwithLRTI', 'numAsthmaAttacks',
       'numAcuteRespEvents', 'numHospEvents', 'postcode_district',

In [25]:
# clean comorbidity and comedication -> NA to 0
com_columns = features.columns[features.columns.str.contains('count')]
for col in com_columns:
    print(col)
    features.loc[features[col].isna(),col] = 0

count_rhinitis
count_cardiovascular
count_heartfailure
count_psoriasis
count_anaphylaxis
count_diabetes
count_ihd
count_anxiety
count_eczema
count_nasalpolyps
count_paracetamol
count_nsaids
count_betablocker


In [26]:
# Clean imd_decile, NA to 0
features.loc[features['imd_decile'].isna(),'imd_decile'] = 0

In [27]:
features.to_csv('../FinalData/cleaned_features_11072023.csv', index_label=False, index=False)

# Outcome

In [28]:
outcomes = pd.read_csv('../FinalData/data_outcomes_10072023.csv')
outcomes = outcomes.iloc[:,1:]
#select only patients included in features
outcomes = outcomes[outcomes.patid.isin(features.patid.values)]
outcomes.shape

(577962, 9)

In [29]:
outcomes["outcome_combined_6months"] = outcomes.apply(lambda x: (x["outcome_3months"] | x["outcome_6months"]), axis=1)
outcomes["outcome_combined_9months"] = outcomes.apply(lambda x: (x["outcome_combined_6months"] | x["outcome_9months"]), axis=1)
outcomes["outcome_combined_12months"] = outcomes.apply(lambda x: (x["outcome_combined_9months"] | x["outcome_12months"]), axis=1)
outcomes["outcome_combined_15months"] = outcomes.apply(lambda x: (x["outcome_combined_12months"] | x["outcome_15months"]), axis=1)
outcomes["outcome_combined_18months"] = outcomes.apply(lambda x: (x["outcome_combined_15months"] | x["outcome_18months"]), axis=1)
outcomes["outcome_combined_24months"] = outcomes.apply(lambda x: (x["outcome_combined_18months"] | x["outcome_24months"]), axis=1)
outcomes.shape

(577962, 15)

In [30]:
outcomes.shape

(577962, 15)

In [31]:
outcomes.columns

Index(['patid', 'outcome_3months', 'outcome_6months', 'outcome_9months',
       'outcome_12months', 'outcome_15months', 'outcome_18months',
       'outcome_21months', 'outcome_24months', 'outcome_combined_6months',
       'outcome_combined_9months', 'outcome_combined_12months',
       'outcome_combined_15months', 'outcome_combined_18months',
       'outcome_combined_24months'],
      dtype='object')

In [32]:
outcomes.to_csv('../FinalData/cleaned_outcomes_11072023.csv', index_label=False, index=False)